Limpando os dados ficticios de venda de um site desenvolvido para ser criado uma analise de dados 

In [ ]:
import pandas as pd
import numpy as np
import random
import re
from datetime import datetime, timedelta


caminho = 'C:/Users/lbrit/Meus Projetos/ANALISE DE DADOS/listprodutosnegaodosuconatural.txt'

dados_limpos = []

with open(caminho, 'r', encoding='utf-8') as f:
    for linha in f:
        # Busca o nome entre aspas e o preço
        nome_match = re.search(r"'(.*?)'", linha)
        preco_match = re.search(r"price:\s*([\d.]+)", linha)
        
        if nome_match and preco_match:
            dados_limpos.append({
                'Produto': nome_match.group(1),
                'Preco': float(preco_match.group(1))
            })

# Criando o DataFrame corretamente
df = pd.DataFrame(dados_limpos)
display(df.loc[1:10])


,Produto,Preco
1,Limonada Suíça,9.0
2,Morango,11.0
3,Manga,9.5
4,Salada de frutas,12.0
5,Açaí,12.0
6,Tapioca Carne Seca c/ Queijo,20.0
7,Tapioca Carne Seca c/ Catupiry,20.0
8,Tapioca Carne Seca c/ Cheddar,20.0
9,Tapioca Carne Seca c/ Requeijao,22.0
10,Tapioca Frango c/ Queijo,18.0


In [2]:
if not df.empty:
    # Nome do arquivo que será criado
    nome_arquivo_excel = 'Analise_Produtos_Negao_Suco.xlsx'
    
    # Salvando em Excel
    # index=False serve para não criar uma coluna de números (0, 1, 2) no Excel
    df.to_excel(nome_arquivo_excel, index=False, engine='openpyxl')
    
    print(f"✅ SUCESSO! O arquivo '{nome_arquivo_excel}' foi salvo na pasta do seu projeto.")
else:
    print("❌ O DataFrame está vazio, por isso não salvei o Excel.")

✅ SUCESSO! O arquivo 'Analise_Produtos_Negao_Suco.xlsx' foi salvo na pasta do seu projeto.


gerando vendas ficticias 

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Usando o df que já existe
df_produtos = df 

# Configurações da simulação
num_vendas = 1000
data_inicial = datetime(2026, 1, 1)
vendas_lista = []

print("Gerando 1.000 vendas fictícias baseadas nos seus produtos reais...")

for i in range(num_vendas):
    # Seleciona um produto aleatório da sua lista limpa
    produto_row = df_produtos.sample().iloc[0]
    
    # Simula uma data aleatória nos últimos 40 dias
    dias_aleatorios = random.randint(0, 40)
    horas_aleatorias = random.randint(8, 20) # Horário de funcionamento
    minutos_aleatorios = random.randint(0, 59)
    
    data_venda = data_inicial + timedelta(days=dias_aleatorios, hours=horas_aleatorias, minutes=minutos_aleatorios)
    
    # Quantidade aleatória (comum pedirem 1 ou 2 itens)
    quantidade = random.choices([1, 2, 3], weights=[0.7, 0.2, 0.1])[0]
    
    vendas_lista.append({
        'Data_Venda': data_venda,
        'Produto': produto_row['Produto'],
        'Preco_Unitario': produto_row['Preco'],
        'Quantidade': quantidade,
        'Total_Venda': produto_row['Preco'] * quantidade,
        'Metodo_Pagamento': random.choice(['Pix', 'Cartão Crédito', 'Cartão Débito', 'Dinheiro']), 
    })

# Criar o DataFrame de Vendas
df_vendas = pd.DataFrame(vendas_lista)

# Ordenar por data para ficar organizado
df_vendas = df_vendas.sort_values(by='Data_Venda')

# Criando a coluna mes
df_vendas['Mes']= np.where(df_vendas['Data_Venda'].dt.month == 1, 'Janeiro', 'Fevereiro')

# Criando a coluna Total Janeiro e Fevereiro
df_vendas['Total Janeiro'] = np.where(df_vendas['Mes'] == 'Janeiro', df_vendas['Total_Venda'], 0)
df_vendas['Total Fevereiro'] = np.where(df_vendas['Mes'] == 'Fevereiro', df_vendas['Total_Venda'], 0)

# Salvar em: (Excel e CSV)
df_vendas.to_csv('vendas_simuladas_negao.csv', index=False, encoding='utf-8-sig')
df_vendas.to_excel('vendas_simuladas_negao.xlsx', index=False)

print(f"✅ SUCESSO! Arquivos gerados com {num_vendas} vendas.")
display(df_vendas.head())

Gerando 1.000 vendas fictícias baseadas nos seus produtos reais...
✅ SUCESSO! Arquivos gerados com 1000 vendas.


,Data_Venda,Produto,Preco_Unitario,Quantidade,Total_Venda,Metodo_Pagamento,Mes,Total Janeiro,Total Fevereiro
79,2026-01-01 08:26:00,Suco Abacaxi,8.0,3,24.0,Cartão Crédito,Janeiro,24.0,0.0
884,2026-01-01 09:54:00,"Suco Beterraba c/ Cenoura, Laranja, Limao e Ge...",8.0,1,8.0,Pix,Janeiro,8.0,0.0
823,2026-01-01 09:59:00,Sanduiche Natural,10.0,3,30.0,Cartão Débito,Janeiro,30.0,0.0
739,2026-01-01 10:10:00,Tapioca Bacon c/ Ovos e Queijo,18.0,1,18.0,Cartão Débito,Janeiro,18.0,0.0
544,2026-01-01 10:14:00,Vitamina Abacate,15.0,1,15.0,Cartão Crédito,Janeiro,15.0,0.0


Destacando os produtos com mais venda e separando eles dos demais

In [ ]:
Tabela = pd.read_excel('vendas_simuladas_negao.xlsx')


Produtosmaisvendidos = Tabela[['Produto', 'Total Janeiro','Total Fevereiro', 'Total_Venda']].groupby('Produto').sum()
display(Produtosmaisvendidos.style.apply(lambda row: ['background-color: green'] * len(row) if row['Total_Venda'] > 600 else [''] * len(row), axis=1).format('R$ {:,.2f}'))

# Top 3 produtos mais vendidos
top3mais = Produtosmaisvendidos[Produtosmaisvendidos['Total_Venda'] > 600]
display(top3mais.style.set_properties().format('R$ {:,.2f}'))

,Total Janeiro,Total Fevereiro,Total_Venda
Produto,,,
Açaí,R$ 156.00,R$ 96.00,R$ 252.00
Fielzao,R$ 324.00,R$ 54.00,R$ 378.00
Limonada Suíça,R$ 216.00,R$ 99.00,R$ 315.00
Manga,R$ 114.00,R$ 38.00,R$ 152.00
Misto Quente,R$ 128.00,R$ 48.00,R$ 176.00
Misto c/ Ovo,R$ 260.00,R$ 70.00,R$ 330.00
Misto c/ Queijo e Peito de Peru,R$ 300.00,R$ 60.00,R$ 360.00
Morango,R$ 253.00,R$ 88.00,R$ 341.00
Pao c/ Linguica,R$ 264.00,R$ 108.00,R$ 372.00


,Total Janeiro,Total Fevereiro,Total_Venda
Produto,,,
Tapioca Bacon c/ Ovos e Queijo,R$ 432.00,R$ 252.00,R$ 684.00
Tapioca Carne Seca c/ Requeijao,R$ 440.00,R$ 220.00,R$ 660.00
"Tapioca Queijo c/ Presunto, Calabresa e Cheddar",R$ 368.00,R$ 256.00,R$ 624.00


Finalizando com o total de cada mês.

In [30]:
totalmes = Tabela[['Total Janeiro','Total Fevereiro']].sum().to_frame(name='Total')
display(totalmes.style.format('R$ {:,.2f}'))

,Total
Total Janeiro,"R$ 13,231.00"
Total Fevereiro,"R$ 4,947.00"
